In [1]:
pip install autokeras -q

     |████████████████████████████████| 174kB 3.9MB/s 
     |████████████████████████████████| 71kB 3.7MB/s 


In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights=None)
print(model(np.random.rand(100, 32, 32, 3)).shape)

(100, 1, 1, 1024)


### Build an AutoML block to select among different DenseNet models

In [3]:
import autokeras as ak
import tensorflow as tf


class DenseNetBlock(ak.Block):
  def build(self, hp, inputs):
    version = hp.Choice("version", ["DenseNet121", "DenseNet169", "DenseNet201"])
    if version == "DenseNet121":
      dense_net_func = tf.keras.applications.DenseNet121
    elif version == "DenseNet169":
      dense_net_func = tf.keras.applications.DenseNet169
    elif version == "DenseNet201":
      dense_net_func = tf.keras.applications.DenseNet201
    return dense_net_func(include_top=False, weights=None)(inputs)

### Build a HyperBlock to select between DenseNet and ResNet

In [4]:
# Model selection block
class SelectionBlock(ak.Block):
  def build(self, hp, inputs):
    if hp.Choice("model", ["densenet", "resnet"]) == "densenet":
        outputs = DenseNetBlock().build(hp, inputs)
    else:
        outputs = ak.ResNetBlock().build(hp, inputs)
    return outputs

In [5]:
# Model selection block with conditional scope
class SelectionBlock(ak.Block):
  def build(self, hp, inputs):
    if hp.Choice("model", ["densenet", "resnet"]) == "densenet":
      with hp.conditional_scope("model", ["densenet"]):
        outputs = DenseNetBlock().build(hp, inputs)
    else:
      with hp.conditional_scope("model", ["resnet"]):
        outputs = ak.ResNetBlock().build(hp, inputs)
    return outputs

### Build model with the customized HyperBlock and conduct search

In [6]:
input_node = ak.ImageInput()
output_node = SelectionBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)
auto_model = ak.AutoModel(input_node, output_node, max_trials=5, overwrite=True)

from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
auto_model.fit(x_train[:100], y_train[:100], epochs=1)

Trial 5 Complete [00h 00m 17s]
val_loss: 2.11826229095459

Best val_loss So Far: 2.11826229095459
Total elapsed time: 00h 01m 39s
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 11s 445ms/step - loss: 2.9181 - accuracy: 0.0829
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [7]:
auto_model.tuner.search_space_summary()

Search space summary
Default search space size: 9
selection_block_1/model (Choice)
{'default': 'densenet', 'conditions': [], 'values': ['densenet', 'resnet'], 'ordered': False}
selection_block_1/dense_net_block_1/version (Choice)
{'default': 'DenseNet121', 'conditions': [{'class_name': 'Parent', 'config': {'name': 'selection_block_1/model', 'values': ['densenet']}}], 'values': ['DenseNet121', 'DenseNet169', 'DenseNet201'], 'ordered': False}
classification_head_1/spatial_reduction_1/reduction_type (Choice)
{'default': 'flatten', 'conditions': [], 'values': ['flatten', 'global_max', 'global_avg'], 'ordered': False}
classification_head_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'adam_weight_decay'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001, 2e-05, 1e-05], 'ordered': True}
selection_